# Outage Severity

**Name(s)**: Kyle Shiroma, 

**Website Link**: (your website link)

In [38]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

In [39]:
import pandas as pd

outage = pd.read_excel('/Users/kyleshiroma/Desktop/outage.xlsx')
outage

,Major power outage events in the continental U.S.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56
0,Time period: January 2000 - July 2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Regions affected: Outages reported in this dat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,variables,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,NaN,1530,2011,12,North Dakota,ND,MRO,West North Central,-0.9,cold,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1536,NaN,1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1537,NaN,1532,2009,8,South Dakota,SD,RFC,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256
1538,NaN,1533,2009,8,South Dakota,SD,MRO,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256


In [40]:
import pandas as pd
import numpy as np
import plotly.express as px

outage = pd.read_excel('/Users/kyleshiroma/Desktop/outage.xlsx', skiprows=5)
outage = outage.drop(index=0).reset_index(drop=True)
outage = outage.drop(columns=['variables'])

numeric_cols = [
    'OUTAGE.DURATION', 'CUSTOMERS.AFFECTED', 'DEMAND.LOSS.MW', 'YEAR', 'MONTH',
    'ANOMALY.LEVEL', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'TOTAL.PRICE',
    'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
    'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN',
    'RES.CUSTOMERS', 'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS',
    'RES.CUST.PCT', 'COM.CUST.PCT', 'IND.CUST.PCT',
    'PC.REALGSP.STATE', 'PC.REALGSP.USA', 'PC.REALGSP.REL', 'PC.REALGSP.CHANGE',
    'UTIL.REALGSP', 'TOTAL.REALGSP', 'UTIL.CONTRI', 'PI.UTIL.OFUSA',
    'POPULATION', 'POPPCT_URBAN', 'POPPCT_UC',
    'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
    'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT', 'PCT_WATER_INLAND'
]
for col in numeric_cols:
    outage[col] = pd.to_numeric(outage[col], errors='coerce')

outage['OUTAGE.START'] = pd.to_datetime(
    outage['OUTAGE.START.DATE'].astype(str) + ' ' + outage['OUTAGE.START.TIME'].astype(str),
    errors='coerce'
)
outage['OUTAGE.RESTORATION'] = pd.to_datetime(
    outage['OUTAGE.RESTORATION.DATE'].astype(str) + ' ' + outage['OUTAGE.RESTORATION.TIME'].astype(str),
    errors='coerce'
)
outage = outage.drop(columns=[
    'OUTAGE.START.DATE', 'OUTAGE.START.TIME',
    'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'
])
outage['YEAR'] = outage['YEAR'].astype('Int64')
outage['MONTH'] = outage['MONTH'].astype('Int64')

# Drop Alaska
outage = outage[outage['U.S._STATE'] != 'Alaska']

# Engineered features
outage['DENSITY_GAP'] = outage['POPDEN_URBAN'] / outage['POPDEN_RURAL'].replace(0, np.nan)
outage['DENSITY_TIER'] = pd.qcut(
    outage['DENSITY_GAP'].dropna(), q=3,
    labels=['Low Gap (more uniform)', 'Medium Gap', 'High Gap (urban-rural divide)']
)
outage['CUST_MISSING'] = outage['CUSTOMERS.AFFECTED'].isna()
outage['SEVERE'] = (outage['OUTAGE.DURATION'] >= outage['OUTAGE.DURATION'].quantile(0.75)).astype('Int64')

print(outage.shape)
print('POPDEN_URBAN' in outage.columns)
outage.head()

(1533, 58)
True


/var/folders/sx/0pn1wtyd3578fh0zgkvc0_rc0000gp/T/ipykernel_93691/1772360349.py:25: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/sx/0pn1wtyd3578fh0zgkvc0_rc0000gp/T/ipykernel_93691/1772360349.py:29: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,...,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION,DENSITY_GAP,DENSITY_TIER,CUST_MISSING,SEVERE
0,1.0,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,...,0.6,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00,125.21978,Medium Gap,False,1
1,2.0,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,...,0.6,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00,125.21978,Medium Gap,True,0
2,3.0,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,...,0.6,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00,125.21978,Medium Gap,False,1
3,4.0,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,...,0.6,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00,125.21978,Medium Gap,False,0
4,5.0,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,...,0.6,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00,125.21978,Medium Gap,False,0


The data is from Purdue university in the form of an excel file. The goal of this project is to do an EDA of the dataset to figure out the severity of outages in a given area. More specifically, what characteristics are associated with higher severity major power outages? This will be steps 1-4 with data cleaning and eda, step 3 is assesment of missningness, and step 4 is hypothesis testing. 5-8 will be modeling based. Framing a prediction problem, baseline model, final model, and fairness analysis. Unsure of what to do for modeling (what question to ask)

## Step 2: Data Cleaning and Exploratory Data Analysis

In [41]:
# Check what columns you actually have
print(outage.columns.tolist())

['OBS', 'YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL', 'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS', 'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT', 'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA', 'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP', 'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN', 'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL', 'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT', 'PCT_WATER_INLAND', 'OUTAGE.START', 'OUTAGE.RESTORATION', 'DENSITY_GAP', 'DENSITY_TIER', 'CUST_MISSING', 'SEVERE']


In [42]:
# Create the density gap feature
outage['DENSITY_GAP'] = outage['POPDEN_URBAN'] / outage['POPDEN_RURAL']

# Quick look at the distribution
print(outage['DENSITY_GAP'].describe())

count    1523.000000
mean      164.409679
std       234.928429
min        12.100492
25%        33.100437
50%        76.216117
75%       160.217105
max      2367.714286
Name: DENSITY_GAP, dtype: float64


In [43]:
# Who's that extreme outlier?
outage.nlargest(5, 'DENSITY_GAP')[['U.S._STATE', 'POPDEN_URBAN', 'POPDEN_RURAL', 'DENSITY_GAP']]

,U.S._STATE,POPDEN_URBAN,POPDEN_RURAL,DENSITY_GAP
1484,Nevada,3314.8,1.4,2367.714286
1485,Nevada,3314.8,1.4,2367.714286
1486,Nevada,3314.8,1.4,2367.714286
1487,Nevada,3314.8,1.4,2367.714286
1488,Nevada,3314.8,1.4,2367.714286


In [44]:
# Alaska shouldn't be in continental U.S. data — drop it
outage = outage[outage['U.S._STATE'] != 'Alaska']
print(f'Rows after dropping Alaska: {len(outage)}')

# Recreate the density gap and tiers
outage['DENSITY_GAP'] = outage['POPDEN_URBAN'] / outage['POPDEN_RURAL'].replace(0, np.nan)
print(outage['DENSITY_GAP'].describe())

Rows after dropping Alaska: 1533
count    1523.000000
mean      164.409679
std       234.928429
min        12.100492
25%        33.100437
50%        76.216117
75%       160.217105
max      2367.714286
Name: DENSITY_GAP, dtype: float64


In [45]:
outage['CUST_MISSING'] = outage['CUSTOMERS.AFFECTED'].isna()

In [46]:
fig_uni1 = px.histogram(
    outage.dropna(subset=['OUTAGE.DURATION']),
    x='OUTAGE.DURATION',
    nbins=80,
    title='Distribution of Outage Duration',
    labels={'OUTAGE.DURATION': 'Duration (minutes)'},
    range_x=[0, 15000],
)
fig_uni1.update_layout(yaxis_title='Count', width=800, height=400)
fig_uni1.show()

In [47]:
# Cause category bar chart — always works and is more relevant
cause_counts = outage['CAUSE.CATEGORY'].value_counts().reset_index()
cause_counts.columns = ['CAUSE.CATEGORY', 'COUNT']

fig_uni2 = px.bar(
    cause_counts,
    x='CAUSE.CATEGORY',
    y='COUNT',
    color='CAUSE.CATEGORY',
    title='Number of Outages by Cause Category',
    labels={'COUNT': 'Number of Outages', 'CAUSE.CATEGORY': 'Cause Category'},
)
fig_uni2.update_layout(showlegend=False, xaxis_tickangle=-30, width=800, height=400)
fig_uni2.show()

In [48]:
fig_bi1 = px.box(
    outage.dropna(subset=['OUTAGE.DURATION', 'DENSITY_TIER']),
    x='DENSITY_TIER',
    y='OUTAGE.DURATION',
    color='CAUSE.CATEGORY',
    log_y=True,
    category_orders={'DENSITY_TIER': [
        'Low Gap (more uniform)', 'Medium Gap', 'High Gap (urban-rural divide)'
    ]},
    title='Outage Duration by Density Gap and Cause',
    labels={
        'OUTAGE.DURATION': 'Duration (minutes)',
        'DENSITY_TIER': 'Density Gap',
        'CAUSE.CATEGORY': 'Cause'
    },
)
fig_bi1.update_layout(width=900, height=500)
fig_bi1.show()

In [49]:
fig_bi2 = px.box(
    outage.dropna(subset=['CUSTOMERS.AFFECTED', 'DENSITY_TIER']),
    x='DENSITY_TIER',
    y='CUSTOMERS.AFFECTED',
    color='DENSITY_TIER',
    log_y=True,
    category_orders={'DENSITY_TIER': [
        'Low Gap (more uniform)', 'Medium Gap', 'High Gap (urban-rural divide)'
    ]},
    title='Customers Affected by Urban-Rural Density Gap',
    labels={
        'CUSTOMERS.AFFECTED': 'Customers Affected',
        'DENSITY_TIER': 'Density Gap'
    },
)
fig_bi2.update_layout(showlegend=False, width=750, height=450)
fig_bi2.show()

## Step 3: Assessment of Missingness

In [50]:
# ============================================================
# STEP 3: ASSESSMENT OF MISSINGNESS
# ============================================================

# --- Cell 1: Check missingness rates ---
missing = outage.isnull().sum()
missing_pct = (missing / len(outage) * 100).round(1)
missing_pct[missing_pct > 0].sort_values(ascending=False).head(10)

HURRICANE.NAMES          95.3
DEMAND.LOSS.MW           46.0
CAUSE.CATEGORY.DETAIL    30.7
CUSTOMERS.AFFECTED       28.9
OUTAGE.RESTORATION        3.7
OUTAGE.DURATION           3.7
RES.SALES                 1.4
COM.SALES                 1.4
COM.PERCEN                1.4
RES.PERCEN                1.4
dtype: float64

In [51]:
# --- Cell 2: Create missingness indicator ---
outage['CUST_MISSING'] = outage['CUSTOMERS.AFFECTED'].isna()
print(f"Missing: {outage['CUST_MISSING'].sum()}")
print(f"Not missing: {(~outage['CUST_MISSING']).sum()}")

Missing: 443
Not missing: 1090


In [52]:
# --- Cell 3: Permutation test — DEPENDS on CAUSE.CATEGORY ---
def tvd(dist1, dist2):
    combined = pd.concat([dist1, dist2], axis=1).fillna(0)
    return (combined.iloc[:, 0] - combined.iloc[:, 1]).abs().sum() / 2

missing_dist = outage[outage['CUST_MISSING']]['CAUSE.CATEGORY'].value_counts(normalize=True)
not_missing_dist = outage[~outage['CUST_MISSING']]['CAUSE.CATEGORY'].value_counts(normalize=True)
observed_tvd = tvd(missing_dist, not_missing_dist)
print(f'Observed TVD: {observed_tvd:.4f}')

labels = outage['CUST_MISSING'].values
causes = outage['CAUSE.CATEGORY'].values

n_reps = 1000
tvds = []
for _ in range(n_reps):
    shuffled = np.random.permutation(labels)
    miss_dist = pd.Series(causes[shuffled]).value_counts(normalize=True)
    not_dist = pd.Series(causes[~shuffled]).value_counts(normalize=True)
    tvds.append(tvd(miss_dist, not_dist))

p_value_cause = np.mean(np.array(tvds) >= observed_tvd)
print(f'P-value: {p_value_cause}')

Observed TVD: 0.5581
P-value: 0.0


In [53]:
# --- Cell 4: Visualize — Cause distribution when missing vs not ---
cause_missing = outage.groupby(['CAUSE.CATEGORY', 'CUST_MISSING']).size().reset_index(name='count')
cause_missing['CUST_MISSING'] = cause_missing['CUST_MISSING'].map({True: 'Missing', False: 'Not Missing'})

totals = cause_missing.groupby('CUST_MISSING')['count'].transform('sum')
cause_missing['proportion'] = cause_missing['count'] / totals

fig_miss1 = px.bar(
    cause_missing,
    x='CAUSE.CATEGORY',
    y='proportion',
    color='CUST_MISSING',
    barmode='group',
    title=f'Distribution of Cause Category by CUSTOMERS.AFFECTED Missingness (TVD={observed_tvd:.3f}, p={p_value_cause})',
    labels={
        'proportion': 'Proportion',
        'CAUSE.CATEGORY': 'Cause Category',
        'CUST_MISSING': 'CUSTOMERS.AFFECTED'
    },
)
fig_miss1.update_layout(xaxis_tickangle=-30, width=850, height=450)
fig_miss1.show()

In [54]:
# --- Cell 5: Empirical distribution — Cause test ---
fig_perm1 = px.histogram(
    x=tvds,
    nbins=50,
    title=f'Permutation Test: Missingness vs CAUSE.CATEGORY (p={p_value_cause})',
    labels={'x': 'Total Variation Distance'},
    opacity=0.7,
)
fig_perm1.add_vline(x=observed_tvd, line_dash='dash', line_color='red',
                    annotation_text=f'Observed TVD: {observed_tvd:.3f}')
fig_perm1.update_layout(width=800, height=400, showlegend=False)
fig_perm1.show()

In [55]:
# --- Cell 6: Permutation test — does NOT depend on COM.PERCEN ---
subset_com = outage.dropna(subset=['COM.PERCEN']).copy()
labels_com = subset_com['CUST_MISSING'].values
com_vals = subset_com['COM.PERCEN'].values

obs_diff_com = abs(com_vals[labels_com].mean() - com_vals[~labels_com].mean())
print(f'Observed diff in mean COM.PERCEN: {obs_diff_com:.4f}')

n_reps = 1000
diffs_com = []
for _ in range(n_reps):
    shuffled = np.random.permutation(labels_com)
    diff = abs(com_vals[shuffled].mean() - com_vals[~shuffled].mean())
    diffs_com.append(diff)

p_value_com = np.mean(np.array(diffs_com) >= obs_diff_com)
print(f'P-value: {p_value_com}')

Observed diff in mean COM.PERCEN: 0.0085
P-value: 0.987


In [56]:
# --- Cell 7: Visualize — COM.PERCEN distribution when missing vs not ---
fig_miss_com = px.histogram(
    subset_com,
    x='COM.PERCEN',
    color='CUST_MISSING',
    barmode='overlay',
    opacity=0.6,
    nbins=40,
    color_discrete_map={True: 'red', False: 'steelblue'},
    title=f'Distribution of COM.PERCEN by CUSTOMERS.AFFECTED Missingness (p={p_value_com})',
    labels={
        'COM.PERCEN': 'Commercial Electricity Consumption (%)',
        'CUST_MISSING': 'CUSTOMERS.AFFECTED Missing'
    },
)
fig_miss_com.update_layout(width=800, height=400)
fig_miss_com.show()

In [57]:
# --- Cell 8: Empirical distribution — COM.PERCEN test ---
fig_perm_com = px.histogram(
    x=diffs_com,
    nbins=50,
    title=f'Permutation Test: Missingness vs COM.PERCEN (p={p_value_com})',
    labels={'x': 'Absolute Difference in Mean COM.PERCEN'},
    opacity=0.7,
)
fig_perm_com.add_vline(x=obs_diff_com, line_dash='dash', line_color='red',
                       annotation_text=f'Observed: {obs_diff_com:.4f}')
fig_perm_com.update_layout(width=800, height=400, showlegend=False)
fig_perm_com.show()

# STEP 4


In [58]:
# ============================================================
# STEP 4: HYPOTHESIS TESTING
# ============================================================

# Question: Do states with a high urban-rural density gap have
# significantly shorter outages than states with a low density gap?

# H0: Mean duration is the same for high-gap and low-gap states
# H1: Low-gap states have higher mean duration than high-gap states

combined = outage.dropna(subset=['OUTAGE.DURATION', 'DENSITY_TIER'])
combined = combined[combined['DENSITY_TIER'].isin([
    'High Gap (urban-rural divide)', 'Low Gap (more uniform)'
])].reset_index(drop=True)

durations = combined['OUTAGE.DURATION'].values
labels = combined['DENSITY_TIER'].values

observed_diff = (
    durations[labels == 'Low Gap (more uniform)'].mean() -
    durations[labels == 'High Gap (urban-rural divide)'].mean()
)
print(f'Observed difference: {observed_diff:.2f} minutes ({observed_diff/60:.2f} hours)')

n_reps = 10000
diffs = []
for _ in range(n_reps):
    shuffled = np.random.permutation(labels)
    low_mean = durations[shuffled == 'Low Gap (more uniform)'].mean()
    high_mean = durations[shuffled == 'High Gap (urban-rural divide)'].mean()
    diffs.append(low_mean - high_mean)

p_value = np.mean(np.array(diffs) >= observed_diff)
print(f'P-value: {p_value}')

Observed difference: 906.08 minutes (15.10 hours)
P-value: 0.0008


In [59]:
# Visualize
fig_hyp = px.histogram(
    x=diffs,
    nbins=50,
    title=f'Permutation Test: Low Gap vs High Gap Duration (p={p_value})',
    labels={'x': 'Difference in Mean Duration (minutes)'},
    opacity=0.7,
)
fig_hyp.add_vline(x=observed_diff, line_dash='dash', line_color='red',
                  annotation_text=f'Observed: {observed_diff:.0f} min')
fig_hyp.update_layout(width=800, height=400, showlegend=False)
fig_hyp.show()

In [60]:
# Quick accuracy checks
print("=== SANITY CHECKS ===")
print(f"Rows: {len(outage)}")
print(f"Columns: {len(outage.columns)}")
print(f"Year range: {outage['YEAR'].min()} - {outage['YEAR'].max()}")
print(f"Alaska present: {'Alaska' in outage['U.S._STATE'].values}")
print(f"DENSITY_GAP nulls: {outage['DENSITY_GAP'].isna().sum()}")
print(f"SEVERE column exists: {'SEVERE' in outage.columns}")
print(f"CUST_MISSING column exists: {'CUST_MISSING' in outage.columns}")
print(f"\nCause counts:\n{outage['CAUSE.CATEGORY'].value_counts()}")
print(f"\nMissing CUSTOMERS.AFFECTED: {outage['CUSTOMERS.AFFECTED'].isna().sum()}")
print(f"\nDensity tier counts:\n{outage['DENSITY_TIER'].value_counts()}")

=== SANITY CHECKS ===
Rows: 1533
Columns: 58
Year range: 2000 - 2016
Alaska present: False
DENSITY_GAP nulls: 10
SEVERE column exists: True
CUST_MISSING column exists: True

Cause counts:
CAUSE.CATEGORY
severe weather                   763
intentional attack               418
system operability disruption    127
public appeal                     69
equipment failure                 59
fuel supply emergency             51
islanding                         46
Name: count, dtype: int64

Missing CUSTOMERS.AFFECTED: 443

Density tier counts:
DENSITY_TIER
Low Gap (more uniform)           594
High Gap (urban-rural divide)    502
Medium Gap                       427
Name: count, dtype: int64


## Step 5: Framing a Prediction Problem

In [61]:
# ============================================================
# STEP 5: FRAMING A PREDICTION PROBLEM
# ============================================================

# Prediction problem: Can we predict whether a power outage will be "severe"
# (top 25% by duration) based on characteristics known at the time of the outage?
#
# This is a BINARY CLASSIFICATION problem.
# Target: SEVERE (1 = top 25% duration, 0 = not)
# Metric: F1 score — because the classes are imbalanced (75/25 split),
# accuracy alone would be misleading (a model that always predicts 0 gets 75%).
# F1 balances precision and recall for the minority (severe) class.
#
# Features must be known at the TIME OF THE OUTAGE — no restoration info,
# no duration, no customers affected (that's leakage).

threshold = outage['OUTAGE.DURATION'].quantile(0.75)
print(f'Severity threshold: {threshold:.0f} minutes ({threshold/60:.1f} hours)')
print(f'\nClass distribution:')
print(outage['SEVERE'].value_counts())
print(f'\nBaseline accuracy (always predict not severe): {1 - outage["SEVERE"].mean():.3f}')

Severity threshold: 2880 minutes (48.0 hours)

Class distribution:
SEVERE
0    1156
1     377
Name: count, dtype: Int64

Baseline accuracy (always predict not severe): 0.754


## Step 6: Baseline Model

In [62]:
# ============================================================
# STEP 6: BASELINE MODEL
# ============================================================

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score

# Baseline uses only 4 simple features:
# - CAUSE.CATEGORY (categorical): strongest predictor from EDA
# - CLIMATE.CATEGORY (categorical): warm/cold/normal
# - NERC.REGION (categorical): grid region
# - MONTH (numeric): seasonality
#
# These are deliberately basic — no engineered features, no tuning.

baseline_features = ['CAUSE.CATEGORY', 'CLIMATE.CATEGORY', 'NERC.REGION', 'MONTH']

model_df = outage.dropna(subset=['SEVERE']).copy()
model_df = model_df.dropna(subset=baseline_features)

X = model_df[baseline_features]
y = model_df['SEVERE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Train size: {len(X_train)}')
print(f'Test size: {len(X_test)}')

Train size: 1220
Test size: 305


## Step 7: Final Model

In [63]:
# ============================================================
# STEP 7: FINAL MODEL
# ============================================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Final model improvements over baseline:
# 1. More features — add demographic, economic, and engineered features
# 2. Better algorithm — Random Forest (ensemble of trees, less overfitting)
# 3. Hyperparameter tuning — GridSearchCV with 5-fold cross-validation
#
# New features and why:
# - ANOMALY.LEVEL: El Nino/La Nina affects weather severity
# - POPULATION: larger states may have different grid resilience
# - POPPCT_URBAN: urban vs rural infrastructure differences
# - TOTAL.CUSTOMERS: grid size as proxy for utility resources
# - TOTAL.PRICE: electricity market conditions
# - PC.REALGSP.STATE: state wealth may affect grid investment
# - DENSITY_GAP: our engineered feature from EDA

final_features = [
    'CAUSE.CATEGORY', 'CLIMATE.CATEGORY', 'NERC.REGION', 'MONTH',
    'ANOMALY.LEVEL', 'POPULATION', 'POPPCT_URBAN',
    'TOTAL.CUSTOMERS', 'TOTAL.PRICE', 'PC.REALGSP.STATE',
    'DENSITY_GAP'
]

model_df2 = outage.dropna(subset=['SEVERE']).copy()
model_df2 = model_df2.dropna(subset=final_features)

X2 = model_df2[final_features]
y2 = model_df2['SEVERE']

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

print(f'Train size: {len(X2_train)}')
print(f'Test size: {len(X2_test)}')
print(f'Features: {len(final_features)}')

Train size: 1201
Test size: 301
Features: 11


In [64]:
# Build preprocessing pipeline
final_preprocess = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['CAUSE.CATEGORY', 'CLIMATE.CATEGORY', 'NERC.REGION']),
    ('num', StandardScaler(), ['MONTH', 'ANOMALY.LEVEL', 'POPULATION', 'POPPCT_URBAN',
                                'TOTAL.CUSTOMERS', 'TOTAL.PRICE', 'PC.REALGSP.STATE',
                                'DENSITY_GAP']),
])

# Hyperparameter grid
param_grid = {
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [5, 10, 20, None],
    'clf__min_samples_split': [2, 5, 10],
}

final_pipe = Pipeline([
    ('preprocess', final_preprocess),
    ('clf', RandomForestClassifier(random_state=42)),
])

grid = GridSearchCV(final_pipe, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(X2_train, y2_train)

print(f'Best params: {grid.best_params_}')
print(f'Best CV F1:  {grid.best_score_:.4f}')

Best params: {'clf__max_depth': 20, 'clf__min_samples_split': 2, 'clf__n_estimators': 200}
Best CV F1:  0.6400


In [65]:
# Evaluate final model on test set
y_pred_final = grid.predict(X2_test)

print('=== FINAL MODEL: Random Forest + GridSearchCV ===')
print(f'Accuracy:  {accuracy_score(y2_test, y_pred_final):.4f}')
print(f'F1 Score:  {f1_score(y2_test, y_pred_final):.4f}')
print(f'\n{classification_report(y2_test, y_pred_final, target_names=["Not Severe", "Severe"])}')

=== FINAL MODEL: Random Forest + GridSearchCV ===
Accuracy:  0.8272
F1 Score:  0.5357

              precision    recall  f1-score   support

  Not Severe       0.89      0.90      0.89       243
      Severe       0.56      0.52      0.54        58

    accuracy                           0.83       301
   macro avg       0.72      0.71      0.71       301
weighted avg       0.82      0.83      0.82       301



In [67]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, f1_score

# ============ STEP 5 ============
threshold = outage['OUTAGE.DURATION'].quantile(0.75)
outage['SEVERE'] = (outage['OUTAGE.DURATION'] >= threshold).astype('Int64')
print(f'Severity threshold: {threshold:.0f} minutes ({threshold/60:.1f} hours)')
print(f'Baseline accuracy: {1 - outage["SEVERE"].mean():.3f}')

# ============ STEP 6: BASELINE ============
baseline_features = ['CAUSE.CATEGORY', 'CLIMATE.CATEGORY', 'NERC.REGION', 'MONTH']

model_df = outage.dropna(subset=['SEVERE']).copy()
model_df = model_df.dropna(subset=baseline_features)

X = model_df[baseline_features]
y = model_df['SEVERE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

baseline_preprocess = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['CAUSE.CATEGORY', 'CLIMATE.CATEGORY', 'NERC.REGION']),
    ('num', StandardScaler(), ['MONTH']),
])

baseline_pipe = Pipeline([
    ('preprocess', baseline_preprocess),
    ('clf', DecisionTreeClassifier(random_state=42)),
])

baseline_pipe.fit(X_train, y_train)
y_pred_base = baseline_pipe.predict(X_test)

print('\n=== BASELINE MODEL: Decision Tree ===')
print(f'Accuracy:  {accuracy_score(y_test, y_pred_base):.4f}')
print(f'F1 Score:  {f1_score(y_test, y_pred_base):.4f}')
print(f'\n{classification_report(y_test, y_pred_base, target_names=["Not Severe", "Severe"])}')

# ============ STEP 7: FINAL MODEL ============
final_features = [
    'CAUSE.CATEGORY', 'CLIMATE.CATEGORY', 'NERC.REGION', 'MONTH',
    'ANOMALY.LEVEL', 'POPULATION', 'POPPCT_URBAN',
    'TOTAL.CUSTOMERS', 'TOTAL.PRICE', 'PC.REALGSP.STATE',
    'DENSITY_GAP'
]

model_df2 = outage.dropna(subset=['SEVERE']).copy()
model_df2 = model_df2.dropna(subset=final_features)

X2 = model_df2[final_features]
y2 = model_df2['SEVERE']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

final_preprocess = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['CAUSE.CATEGORY', 'CLIMATE.CATEGORY', 'NERC.REGION']),
    ('num', StandardScaler(), ['MONTH', 'ANOMALY.LEVEL', 'POPULATION', 'POPPCT_URBAN',
                                'TOTAL.CUSTOMERS', 'TOTAL.PRICE', 'PC.REALGSP.STATE',
                                'DENSITY_GAP']),
])

param_grid = {
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [5, 10, 20, None],
    'clf__min_samples_split': [2, 5, 10],
}

final_pipe = Pipeline([
    ('preprocess', final_preprocess),
    ('clf', RandomForestClassifier(random_state=42)),
])

grid = GridSearchCV(final_pipe, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(X2_train, y2_train)
y_pred_final = grid.predict(X2_test)

print('\n=== FINAL MODEL: Random Forest + GridSearchCV ===')
print(f'Best params: {grid.best_params_}')
print(f'Accuracy:  {accuracy_score(y2_test, y_pred_final):.4f}')
print(f'F1 Score:  {f1_score(y2_test, y_pred_final):.4f}')
print(f'\n{classification_report(y2_test, y_pred_final, target_names=["Not Severe", "Severe"])}')

# ============ COMPARISON ============
print('\n=== MODEL COMPARISON ===')
base_f1 = f1_score(y_test, y_pred_base)
final_f1 = f1_score(y2_test, y_pred_final)
print(f'Baseline F1:  {base_f1:.4f}')
print(f'Final F1:     {final_f1:.4f}')
print(f'Improvement:  {final_f1 - base_f1:.4f}')

Severity threshold: 2880 minutes (48.0 hours)
Baseline accuracy: 0.754

=== BASELINE MODEL: Decision Tree ===
Accuracy:  0.7443
F1 Score:  0.4429

              precision    recall  f1-score   support

  Not Severe       0.83      0.84      0.83       234
      Severe       0.45      0.44      0.44        71

    accuracy                           0.74       305
   macro avg       0.64      0.64      0.64       305
weighted avg       0.74      0.74      0.74       305


=== FINAL MODEL: Random Forest + GridSearchCV ===
Best params: {'clf__max_depth': 20, 'clf__min_samples_split': 2, 'clf__n_estimators': 200}
Accuracy:  0.8272
F1 Score:  0.5357

              precision    recall  f1-score   support

  Not Severe       0.89      0.90      0.89       243
      Severe       0.56      0.52      0.54        58

    accuracy                           0.83       301
   macro avg       0.72      0.71      0.71       301
weighted avg       0.82      0.83      0.82       301


=== MODEL COMPARISO

## Step 8: Fairness Analysis

In [69]:
# ============ STEP 8: FAIRNESS ANALYSIS ============
cold_mask = X2_test['CLIMATE.CATEGORY'] == 'cold'
warm_mask = X2_test['CLIMATE.CATEGORY'] == 'warm'

f1_cold = f1_score(y2_test[cold_mask], y_pred_final[cold_mask], zero_division=0)
f1_warm = f1_score(y2_test[warm_mask], y_pred_final[warm_mask], zero_division=0)
observed_f1_diff = f1_cold - f1_warm

print(f'F1 (cold climate):  {f1_cold:.4f}')
print(f'F1 (warm climate):  {f1_warm:.4f}')
print(f'Observed F1 diff:   {observed_f1_diff:.4f}')

cold_warm_mask = cold_mask | warm_mask
y_sub = y2_test[cold_warm_mask].reset_index(drop=True)
pred_sub = pd.Series(y_pred_final[cold_warm_mask]).reset_index(drop=True)
groups = X2_test.loc[cold_warm_mask, 'CLIMATE.CATEGORY'].reset_index(drop=True)

n_reps = 1000
perm_diffs = []
for _ in range(n_reps):
    shuffled_groups = np.random.permutation(groups.values)
    cold_shuf = shuffled_groups == 'cold'
    warm_shuf = shuffled_groups == 'warm'
    f1_c = f1_score(y_sub[cold_shuf], pred_sub[cold_shuf], zero_division=0)
    f1_w = f1_score(y_sub[warm_shuf], pred_sub[warm_shuf], zero_division=0)
    perm_diffs.append(f1_c - f1_w)

p_value_fair = np.mean(np.abs(np.array(perm_diffs)) >= np.abs(observed_f1_diff))
print(f'\nPermutation test p-value: {p_value_fair}')

if p_value_fair < 0.05:
    print('Conclusion: Reject H0 — model performs significantly differently across climate categories.')
else:
    print('Conclusion: Fail to reject H0 — no significant unfairness detected.')

fig_fair = px.histogram(
    x=perm_diffs,
    nbins=50,
    title=f'Fairness Test: F1 Parity Across Climate Categories (p={p_value_fair})',
    labels={'x': 'Difference in F1 (Cold - Warm)'},
    opacity=0.7,
)
fig_fair.add_vline(x=observed_f1_diff, line_dash='dash', line_color='red',
                   annotation_text=f'Observed: {observed_f1_diff:.4f}')
fig_fair.update_layout(width=800, height=400, showlegend=False)
fig_fair.show()

F1 (cold climate):  0.6207
F1 (warm climate):  0.6154
Observed F1 diff:   0.0053

Permutation test p-value: 0.999
Conclusion: Fail to reject H0 — no significant unfairness detected.
